In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
from sebm_mnist.data import load_data
from sebm_mnist.modules.sgld import SGLD_sampler
from sebm_mnist.modules.energy_function_anatomy import Energy_function

CUDA = torch.cuda.is_available()
if CUDA:
    DEVICE = torch.device('cuda:0')
print('torch:', torch.__version__, 'CUDA:', CUDA)
## EBM hyper-parameters
sgld_noise_std = 0.001
clipping_sgld = False
# LOAD_VERSION = 'ebm-lr=5.00E-05-data-std=7.50E-03-sgld-steps=4.00E+01-size=1.00E+00-std=5.00E-03-reg=1.00E-03-clip=False'
LOAD_VERSION = 'ebm-lr=5.00E-05-data-std=7.50E-03-sgld-steps=4.00E+01-size=1.00E+00-std=5.00E-03-reg=1.00E-03-clip=False'
# LOAD_VERSION = 'ebm-lr=5.00E-05-data-std=1.00E-02-sgld-steps=4.00E+01-size=1.00E+00-std=5.00E-03-reg=1.00E-02-clip=False'
print('Load trained energy function...')
ef = Energy_function()
if CUDA:
    ef.cuda().to(DEVICE)   
# ef.load_state_dict(torch.load('../weights/ef-%s' % LOAD_VERSION))
# for p in ef.parameters():
#     p.requires_grad = False
print('Initialize SGLD sampler...')
sgld_sampler = SGLD_sampler(sgld_noise_std, clipping_sgld, CUDA, DEVICE)

torch: 1.3.0 CUDA: True
Load trained energy function...
Initialize SGLD sampler...


In [2]:
## data directory
print('Load MNIST dataset...')
DATA_DIR = '/home/hao/Research/sebm_data/'
train_data, test_data = load_data(DATA_DIR, 10)
for (images, _) in train_data:
    break
if CUDA:
    images = images.cuda().to(DEVICE)
E = ef.forward(images)

Load MNIST dataset...


In [3]:
E.shape

torch.Size([10, 1, 1, 1])

In [ ]:
visual_sample_size = 10
ebm_images = sgld_sampler.sgld_update(ef=ef, 
                                      batch_size=visual_sample_size, 
                                      pixels_size=28, 
                                      num_steps=1000, 
                                      step_size=1,
                                      buffer_size=None,
                                      buffer_percent=None,
                                      persistent=False)
ebm_images = ebm_images.squeeze(0).cpu().view(visual_sample_size, 28, 28)
ebm_images = torch.clamp(ebm_images, min=-1, max=1) * 0.5 + 0.5

In [ ]:
gs = gridspec.GridSpec(int(visual_sample_size/10), 10)
gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0, hspace=0)
fig = plt.figure(figsize=(10, 10*int(visual_sample_size/10)/ 10))
for i in range(visual_sample_size):
    ax = fig.add_subplot(gs[int(i/10), i%10])
    ax.imshow(ebm_images[i], cmap='gray', vmin=0, vmax=1.0)
    ax.set_xticks([])
    ax.set_yticks([])
# plt.savefig('ghost.svg')